https://chat.openai.com/share/f0c24d12-963d-4525-aa10-7d683a304579


In [ ]:
!pip install alpaca-py

In [ ]:
!pip install torch

In [ ]:
!set APCA_API_BASE_URL=https://paper-api.alpaca.markets

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
from alpaca.trading.client import *
from alpaca.trading.requests import *
from alpaca.data.historical import StockHistoricalDataClient, CryptoHistoricalDataClient
from alpaca.data.requests import StockBarsRequest, CryptoBarsRequest
from alpaca.data.live import CryptoDataStream, StockDataStream
from alpaca.trading.enums import OrderSide, TimeInForce, QueryOrderStatus
from alpaca.data.timeframe import TimeFrame
from datetime import datetime

# Plan:
collect data D from historical api,
process it to be useful,
train model X with data.
D can be any data spaced apart by any type of interval.

## Goal:

X needs to make decisions of buy and selling stocks as accurately as possible.
until Day Trading is permited, X must make trades of minimum a day long. X must not risk large amounts. X must give priorities to stocks which have higher success ratios to allocate more money to them.

D must be information dense with respect to timeframe. D must not be redundant to conserve speed.

## Future:

When I have access to a powerful enough GPU, train X on as many stocks as possible. Then finetune copies of X for multiple stocks. Finally, cash in.

In [ ]:
api_key = "*"
secret_key = "*"
alpaca_client = TradingClient(api_key,secret_key,paper = True)


def sumbit_order(buy_or_sell,symbol,money):
  order_data = MarketOrderRequest(
    symbol = symbol,
    notional = money,
    side = OrderSide.BUY if buy_or_sell=='buy' else OrderSide.SELL,
    time_in_force = TimeInForce.DAY
  )
  alpaca_client.submit_order(order_data)

In [ ]:
r=CryptoBarsRequest(symbol_or_symbols="BTC/USD",timeframe=TimeFrame.Minute,start=datetime(2023,8,25),end=datetime(2023,8,26))
b=historical_data_client.get_crypto_bars(r).df

In [ ]:
b

In [ ]:
historical_data_client = CryptoHistoricalDataClient(api_key,secret_key)

#symbol -> which stock to buy
#seq_length -> how many days are being used to predict next day
def getHistoricalData(symbol, seq_length, start_date, end_date, time_frame=TimeFrame.Minute):
  request_params = CryptoBarsRequest(
                        symbol_or_symbols = symbol,
                        timeframe = time_frame,
                        start = start_date,
                        end = end_date
                 )
  bars = historical_data_client.get_crypto_bars(request_params).df
  open_max=np.max(bars.open)
  high_max=np.max(bars.high)
  low_max=np.max(bars.low)
  close_max=np.max(bars.close)
  volume_max=np.max(bars.volume)

  data = np.array([[bars.open[i]/open_max, bars.high[i]/high_max, bars.low[i]/low_max, bars.close[i]/close_max, bars.volume[i]/volume_max]
                   for i in range(len(bars))])
  data = data.astype(np.float32)

  sequences = []
  for i in range(len(data) - seq_length):
      seq = data[i:i + seq_length]
      label = data[i + seq_length]
      sequences.append((seq, label))

  return sequences, open_max


In [ ]:
data_stream = CryptoDataStream(api_key, secret_key)

async def quote_data_handler(data):
    # quote data will arrive here
    print(data)

data_stream.subscribe_bars(quote_data_handler,"BTC/USD")


In [ ]:
#data_stream.unsubscribe_quotes("BTC/USD")


In [ ]:
data_stream.close()

<coroutine object BaseStream.close at 0x7842aedf5bd0>

In [ ]:
try:
  await data_stream._run_forever()
except KeyboardInterrupt:
  print("keyboard interrupt, bye")
  pass
data_stream.close()

In [ ]:
sequences, data_max = getHistoricalData("BTC/USD",4,start_date=datetime(2022,7,1),end_date=datetime(2022,7,11))
train_size = int(0.8 * len(sequences))
train_data = sequences[:train_size]
test_data = sequences[train_size:]
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

In [ ]:
len(sequences)

14393

In [ ]:
torch.

device(type='xpu')

In [ ]:
class LSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, (h,c) = self.lstm(x)

        out = self.fc(out[:, -1, :])
        return out

In [ ]:
input_size = 5 #ohlcv
hidden_size = 50
num_layers = 2
output_size = 1
num_epochs = 100
learning_rate = 0.001

model = LSTM(input_size, hidden_size, num_layers, output_size)
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
a=0
for seq, labels in train_loader:
  a=seq
  break

In [ ]:
a.shape

torch.Size([32, 4, 5])

In [ ]:
# Training
losses=[]
def train():
  for epoch in range(num_epochs):
    for seq, labels in train_loader:
      optimizer.zero_grad()
      outputs = model(seq)
      loss = loss_fn(outputs[:,0], labels[:,0])
      losses.append(loss.detach().numpy())
      loss.backward()
      optimizer.step()


    if (epoch + 1) % 10 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item()}')
  plt.plot(losses)

In [ ]:
# Evaluating
def eval():
  model.eval()
  eval_loss=0
  eval_n=0
  with torch.no_grad():
    for seq, labels in test_loader:
      outputs = model(seq)
      loss = loss_fn(outputs[:,0], labels[:,0])
      n=len(seq)
      eval_n+=n
      eval_loss+=loss*n

    eval_loss/=eval_n
    print(f'Loss: {eval_loss.item()}')

In [ ]:
# Testing
def test(n=1):
  model.eval()
  with torch.no_grad():
    for seq, labels in test_loader:
      outputs = model(seq)
      for o,l in zip(outputs.numpy(),labels):
        n-=1
        if n<0:
          return
        print(f'Output: {o[0]*data_max}',f'Label: {l.numpy()[0]*data_max}','\n')


In [ ]:
test(100)

In [ ]:
for name, params in model.named_parameters():
  print(name, params)

In [ ]:
def eval(model):
  model.eval()
  model.

In [ ]:
x=torch.tensor([1,0,0])
x.numpy()[0]

1